# 🧱 Montaje (Mount) en Azure Databricks — Actualizado 2025

## 📘 Concepto
El **montaje (mount)** en **Azure Databricks** consiste en crear un alias o punto de montaje dentro del sistema de archivos de Databricks (DBFS — *Databricks File System*) que apunta a una ubicación de almacenamiento en la nube, como **Azure Blob Storage** o **Azure Data Lake Storage Gen2**.  

Este alias permite acceder a los datos usando rutas simples como `/mnt/<nombre>` en lugar de usar las largas URLs de almacenamiento (por ejemplo, `abfss://...`).  

> 📅 **Nota actual (2025):**  
> El uso de montajes se considera una **funcionalidad heredada** (*legacy access pattern*).  
> Databricks y Microsoft recomiendan actualmente utilizar **Unity Catalog** para la gestión de accesos y permisos sobre el almacenamiento.  
>  
> 🔗 [Referencia oficial — Microsoft Learn](https://learn.microsoft.com/en-us/azure/databricks/connect/storage/azure-storage)

---

## 💻 Ejemplo de uso en un Notebook de Databricks

```python
# Montar un contenedor de Azure Data Lake Storage Gen2 en Databricks

dbutils.fs.mount(
    source = "abfss://<contenedor>@<cuenta-almacenamiento>.dfs.core.windows.net/",
    mount_point = "/mnt/<nombre_del_montaje>",
    extra_configs = {
        "fs.azure.account.auth.type.<cuenta-almacenamiento>.dfs.core.windows.net": "OAuth",
        "fs.azure.account.oauth.provider.type.<cuenta-almacenamiento>.dfs.core.windows.net": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
        "fs.azure.account.oauth2.client.id.<cuenta-almacenamiento>.dfs.core.windows.net": "<application-id>",
        "fs.azure.account.oauth2.client.secret.<cuenta-almacenamiento>.dfs.core.windows.net": dbutils.secrets.get(scope="<scope>", key="<key>"),
        "fs.azure.account.oauth2.client.endpoint.<cuenta-almacenamiento>.dfs.core.windows.net": "https://login.microsoftonline.com/<directory-id>/oauth2/token"
    }
)


# 🧱 Montar (Mount) Azure Data Lake Storage Gen2 en Databricks mediante Service Principal

## 🔹 Descripción general
Este proceso permite conectar Azure Databricks con Azure Data Lake Storage (ADLS Gen2) de forma segura usando un **Service Principal** registrado en **Microsoft Entra ID (Azure AD)**.  
De esta manera, podrás acceder a los datos desde rutas locales (`/mnt/...`) en el sistema de archivos de Databricks (DBFS).

---

## 🚀 Pasos para realizar el montaje

### 1️⃣ Obtener credenciales seguras desde Azure Key Vault
Antes de montar el almacenamiento, necesitas los valores siguientes, que puedes guardar como secretos en **Azure Key Vault** o en un **Databricks Secret Scope**:

- **Client ID (application id)**  
- **Tenant ID (directory id)**  
- **Client Secret (password del Service Principal)**  

> 💡 *Recomendación:* Guarda estos valores en Azure Key Vault e intégralo con Databricks para no exponer credenciales directamente en el código.  
> 🔗 [Integrar Key Vault con Databricks](https://learn.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes)

---

### 2️⃣ Configurar variables en Databricks (Service Principal)
Define los parámetros necesarios para la autenticación con tu Service Principal.

```python
storage_account_name = "<nombre_cuenta_storage>"
container_name = "<nombre_contenedor>"
mount_point = "/mnt/<nombre_montaje>"

client_id = dbutils.secrets.get(scope="<scope>", key="client-id")
tenant_id = dbutils.secrets.get(scope="<scope>", key="tenant-id")
client_secret = dbutils.secrets.get(scope="<scope>", key="client-secret")


In [0]:
dbutils.secrets.listScopes()

In [0]:
dbutils.secrets.list("sc-udemy-course")

In [0]:
client_id=dbutils.secrets.get(scope="sc-udemy-course",key="client-id-proyect-joseph")
tenan_id=dbutils.secrets.get(scope="sc-udemy-course",key="tenan-id-proyect-joseph")
client_secret=dbutils.secrets.get(scope="sc-udemy-course",key="secret-client-proyect-joseph")

In [0]:
name_storage="adlsg2proyectudemy"

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenan_id}/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = f"abfss://bronze@{name_storage}.dfs.core.windows.net/",
  mount_point = f"/mnt/{name_storage}/bronze",  #una buena practica es nombre storage / nombre de container
  extra_configs = configs)

In [0]:
df_categorias=spark.read.csv(f"/mnt/adlsg2proyectudemy/bronze/categorias.csv", header=True)

In [0]:
df_categorias.limit(5).display()

In [0]:
#para ver los montajes
display(dbutils.fs.mounts())

In [0]:
#PARA PODER DESMONTAR
dbutils.fs.unmount("/mnt/adlsg2proyectudemy/bronze")

In [0]:
#para ver los montajes
display(dbutils.fs.mounts())

# 🧩 UDF (User Defined Functions) en Spark / Databricks

## 📘 Concepto
Las **UDF (User Defined Functions)** son **funciones definidas por el usuario** que permiten extender las capacidades de **Spark SQL o PySpark**.  
Se utilizan cuando las funciones integradas de Spark no son suficientes para realizar una operación específica sobre los datos.

Con una UDF, puedes crear tu propia lógica en **Python, Scala o Java**, y aplicarla a columnas de un DataFrame como si fuera una función nativa de Spark.

---

## 💻 Ejemplo en PySpark

```python
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Definir función Python
def mayusculas(texto):
    return texto.upper() if texto else None

# Registrar como UDF en Spark
udf_mayusculas = udf(mayusculas, StringType())

# Aplicar la UDF a una columna
df = df.withColumn("nombre_mayus", udf_mayusculas(df["nombre"]))
display(df)


#### MONTAJE DE TODOS LOS CANTAINERS DE MI AZURE DATA LAKE

In [0]:
[mount.mountPoint for mount in dbutils.fs.mounts()]

In [0]:
def mount_adls_udf(client_id,tenan_id,client_secret,name_storage,name_container):
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenan_id}/oauth2/token"}
    
    #si ya existe el montaje lo vamos a desmontar
    if any(mount.mountPoint == f"/mnt/{name_storage}/{name_container}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{name_storage}/{name_container}")

# Optionally, you can add <directory-name> to the source URI of your mount point.
    dbutils.fs.mount(
        source = f"abfss://{name_container}@{name_storage}.dfs.core.windows.net/",
         mount_point = f"/mnt/{name_storage}/{name_container}",  #una buena practica es nombre storage / nombre de container
        extra_configs = configs)
    
    display(dbutils.fs.mounts())
    


In [0]:
#utilizando la funcion
mount_adls_udf(client_id,tenan_id,client_secret,"adlsg2proyectudemy","silver")


In [0]:
mount_adls_udf(client_id,tenan_id,client_secret,"adlsg2proyectudemy","gold")